# Cross dataset OHLCV Data QA

This notebook is used to perform quality assurance of cross dataset OHLCV data.
As displayed below, the notebook assumes environment variables for the data QA parameters. The intended usage
is via invoke target `dev_scripts.lib_tasks_data_qa.run_cross_dataset_qa_notebook`

## Imports and logging

In [1]:
import logging

import pandas as pd

import core.config as cconfig
import data_schema.dataset_schema_utils as dsdascut
import helpers.hdbg as hdbg
import helpers.henv as henv
import helpers.hio as hio
import helpers.hprint as hprint
import im_v2.common.data.client.im_raw_data_client as imvcdcimrdc
import im_v2.common.data.qa.dataset_validator as imvcdqdava
import im_v2.common.data.qa.qa_check as imvcdqqach
import im_v2.common.universe.universe as imvcounun

/app/amp/core/statistics/random_samples.py:12: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm


### Logging

In [2]:
hdbg.init_logger(verbosity=logging.INFO)

_LOG = logging.getLogger(__name__)

_LOG.info("%s", henv.get_system_signature()[0])

hprint.config_notebook()

INFO: > cmd='/venv/lib/python3.9/site-packages/ipykernel_launcher.py -f /home/.local/share/jupyter/runtime/kernel-a75e6330-5c81-477d-9d24-e9ef1f1ee8a5.json'
-----------------------------------------------------------------------------
This code is not in sync with the container:
code_version='1.10.0' != container_version='1.11.0'
-----------------------------------------------------------------------------
You need to:
- merge origin/master into your branch with `invoke git_merge_master`
- pull the latest container with `invoke docker_pull`
INFO  # Git
  branch_name='CmampTask6683_OHLCV_QA_improvement'
  hash='4dceca867'
  # Last commits:
    * 4dceca867 sonaalKant checkpoint                                                        (85 seconds ago) Wed Jan 24 19:35:37 2024  (HEAD -> CmampTask6683_OHLCV_QA_improvement, origin/CmampTask6683_OHLCV_QA_improvement)
    *   9a3e4a273 sonaalKant resolve conflicts                                                 (    7 days ago) Wed Jan 17 09:30:

## QA parameters

To assist debugging you can override any of the parameters after its loaded and rerun QA

In [3]:
env_var_name = "CK_DATA_RECONCILIATION_CONFIG"
config = cconfig.Config.from_env_var(env_var_name)

WARNING Environment variable 'CK_DATA_RECONCILIATION_CONFIG' not defined: no config retrieved


In [4]:
if config:
    config = config.to_dict()
else:
    config = {
        "stage": "preprod",
        "start_timestamp": "2024-01-03T20:12:00+00:00",
        "end_timestamp": "2024-01-03T20:50:00+00:00",
        "aws_profile": "ck",
        "dataset_signature1": "realtime.airflow.downloaded_1min.postgres.ohlcv.futures.v7_3.ccxt.binance.v1_0_0",
        "dataset_signature2": "periodic_daily.airflow.downloaded_1min.parquet.ohlcv.futures.v7_3.ccxt.binance.v1_0_0",
        "bid_ask_accuracy": 1,
    }
# bid_ask_accuracy needs to be cast to int if its defined
config["bid_ask_accuracy"] = (
    int(config["bid_ask_accuracy"]) if config["bid_ask_accuracy"] else None
)
# bid_ask_depth needs to be cast to int if its defined
# config["bid_ask_depth"] = int(config["bid_ask_depth"]) if config["bid_ask_depth"] else None
config

{'stage': 'preprod',
 'start_timestamp': '2024-01-03T20:12:00+00:00',
 'end_timestamp': '2024-01-03T20:50:00+00:00',
 'aws_profile': 'ck',
 'dataset_signature1': 'realtime.airflow.downloaded_1min.postgres.ohlcv.futures.v7_3.ccxt.binance.v1_0_0',
 'dataset_signature2': 'periodic_daily.airflow.downloaded_1min.parquet.ohlcv.futures.v7_3.ccxt.binance.v1_0_0',
 'bid_ask_accuracy': 1}

### Parse dataset signature

1. Load dataset schema
2. Validate dataset signature
3. Parse dataset attributes to drive some of the QA configuration

In [5]:
dataset_schema = dsdascut.get_dataset_schema()
dsdascut.validate_dataset_signature(config["dataset_signature1"], dataset_schema)
dsdascut.validate_dataset_signature(config["dataset_signature2"], dataset_schema)

INFO  Loading dataset schema file: /app/amp/data_schema/dataset_schema_versions/dataset_schema_v3.json
INFO  Loaded dataset schema version v3


True

In [6]:
dataset_signature_as_dict1 = dsdascut.parse_dataset_signature_to_args(
    config["dataset_signature1"], dataset_schema
)
dataset_signature_as_dict1

{'download_mode': 'realtime',
 'downloading_entity': 'airflow',
 'action_tag': 'downloaded_1min',
 'data_format': 'postgres',
 'data_type': 'ohlcv',
 'asset_type': 'futures',
 'universe': 'v7_3',
 'vendor': 'ccxt',
 'exchange_id': 'binance',
 'version': 'v1_0_0'}

In [7]:
dataset_signature_as_dict2 = dsdascut.parse_dataset_signature_to_args(
    config["dataset_signature2"], dataset_schema
)
dataset_signature_as_dict2

{'download_mode': 'periodic_daily',
 'downloading_entity': 'airflow',
 'action_tag': 'downloaded_1min',
 'data_format': 'parquet',
 'data_type': 'ohlcv',
 'asset_type': 'futures',
 'universe': 'v7_3',
 'vendor': 'ccxt',
 'exchange_id': 'binance',
 'version': 'v1_0_0'}

## Load Data

TODO(Juraj): At the moment assume that first dataset argument is a DB dataset and second is from S3 because of small preprocessing operations needed before performing QA

### First dataset

In [8]:
raw_data_client = imvcdcimrdc.RawDataReader(
    config["dataset_signature1"], stage=config["stage"]
)
data1 = raw_data_client.read_data(
    pd.Timestamp(config["start_timestamp"]), pd.Timestamp(config["end_timestamp"])
)

INFO  Loading dataset schema file: /app/amp/data_schema/dataset_schema_versions/dataset_schema_v3.json
INFO  Loaded dataset schema version v3
INFO  Loading dataset schema file: /app/amp/data_schema/dataset_schema_versions/dataset_schema_v3.json
INFO  Loaded dataset schema version v3
INFO  Unable to fetch DB credentials from environment variables: 
	'POSTGRES_HOST'
	Attempting env file method.
INFO  Unable to fetch DB credentials from env file: 
	
################################################################################
* Failed assertion *
'preprod' in '['local', 'dev', 'prod']'
################################################################################

	Attempting AWS SecretsManager method.
INFO  Fetching secret: preprod.im_data_db
INFO  Created preprod DB connection: 
 None
INFO  Enabled connection to the `ccxt_ohlcv_futures` DB table
INFO  Executing query: 
	SELECT * FROM ccxt_ohlcv_futures WHERE timestamp >= 1704312720000 AND timestamp <= 1704315000000 AND exchange_id 

/app/amp/helpers/hsql_implementation.py:582: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query(query, connection)


In [9]:
data1.head()

,id,timestamp,open,high,low,close,volume,currency_pair,exchange_id,end_download_timestamp,knowledge_timestamp
0,436454641,1704312720000,2209.3400,2209.3500,2207.4900,2207.5000,448.182,ETH_USDT,binance,2024-01-03 20:13:10.020069+00:00,2024-01-03 20:13:10.044211+00:00
1,436454642,1704312720000,42613.6000,42613.7000,42574.1000,42574.1000,76.916,BTC_USDT,binance,2024-01-03 20:13:10.020484+00:00,2024-01-03 20:13:10.044211+00:00
2,436454643,1704312720000,0.5088,0.5088,0.5080,0.5081,91672.000,SAND_USDT,binance,2024-01-03 20:13:10.020835+00:00,2024-01-03 20:13:10.044211+00:00
3,436454644,1704312720000,0.6235,0.6235,0.6201,0.6201,111169.000,STORJ_USDT,binance,2024-01-03 20:13:10.021199+00:00,2024-01-03 20:13:10.044211+00:00
4,436454645,1704312720000,0.3087,0.3088,0.3084,0.3084,114903.000,GMT_USDT,binance,2024-01-03 20:13:10.021623+00:00,2024-01-03 20:13:10.044211+00:00


### Second dataset

In [10]:
raw_data_client = imvcdcimrdc.RawDataReader(
    config["dataset_signature2"], stage=config["stage"]
)
data2 = raw_data_client.read_data(
    pd.Timestamp(config["start_timestamp"]), pd.Timestamp(config["end_timestamp"])
)
data2 = data2.reset_index(drop=True)

INFO  Loading dataset schema file: /app/amp/data_schema/dataset_schema_versions/dataset_schema_v3.json
INFO  Loaded dataset schema version v3
INFO  Loading dataset schema file: /app/amp/data_schema/dataset_schema_versions/dataset_schema_v3.json
INFO  Loaded dataset schema version v3
INFO  Loading dataset schema file: /app/amp/data_schema/dataset_schema_versions/dataset_schema_v3.json
INFO  Loaded dataset schema version v3
INFO  Loading dataset schema file: /app/amp/data_schema/dataset_schema_versions/dataset_schema_v3.json
INFO  Loaded dataset schema version v3


In [11]:
data2.head()

,timestamp,open,high,low,close,volume,exchange_id,knowledge_timestamp,currency_pair,year,month
0,1704312720000,1.452,1.452,1.450,1.451,43148.0,binance,2024-01-04 01:23:12.362895+00:00,APE_USDT,2024,1
1,1704312780000,1.451,1.453,1.450,1.452,21826.0,binance,2024-01-04 01:23:12.362895+00:00,APE_USDT,2024,1
2,1704312840000,1.451,1.453,1.450,1.453,12976.0,binance,2024-01-04 01:23:12.362895+00:00,APE_USDT,2024,1
3,1704312900000,1.452,1.456,1.452,1.456,38907.0,binance,2024-01-04 01:23:12.362895+00:00,APE_USDT,2024,1
4,1704312960000,1.455,1.458,1.455,1.457,55444.0,binance,2024-01-04 01:23:12.362895+00:00,APE_USDT,2024,1


### Preprocess raw data
- remove columns unimportant for QA
- remove duplicates

In [12]:
cols_to_keep = [
    "timestamp",
    "open",
    "high",
    "low",
    "close",
    "volume",
    "currency_pair",
    "exchange_id",
]
data1 = data1[cols_to_keep].sort_values(
    ["currency_pair", "timestamp"], ascending=True, ignore_index=True
)
data2 = data2[cols_to_keep].sort_values(
    ["currency_pair", "timestamp"], ascending=True, ignore_index=True
)

In [13]:
data1 = data1.drop_duplicates()
data2 = data2.drop_duplicates()

In [14]:
data1.head()

,timestamp,open,high,low,close,volume,currency_pair,exchange_id
0,1704312720000,1.452,1.452,1.450,1.451,43148.0,APE_USDT,binance
1,1704312780000,1.451,1.453,1.450,1.452,21826.0,APE_USDT,binance
2,1704312840000,1.451,1.453,1.450,1.453,12976.0,APE_USDT,binance
3,1704312900000,1.452,1.456,1.452,1.456,38907.0,APE_USDT,binance
4,1704312960000,1.455,1.458,1.455,1.457,55444.0,APE_USDT,binance


In [15]:
data2.head()

,timestamp,open,high,low,close,volume,currency_pair,exchange_id
0,1704312720000,1.452,1.452,1.450,1.451,43148.0,APE_USDT,binance
1,1704312780000,1.451,1.453,1.450,1.452,21826.0,APE_USDT,binance
2,1704312840000,1.451,1.453,1.450,1.453,12976.0,APE_USDT,binance
3,1704312900000,1.452,1.456,1.452,1.456,38907.0,APE_USDT,binance
4,1704312960000,1.455,1.458,1.455,1.457,55444.0,APE_USDT,binance


## Initialize QA checks

### Single dataset checks

In [16]:
datasets = [data1, data2]
signatures = [dataset_signature_as_dict1, dataset_signature_as_dict2]
qa_check_lists = []

In [17]:
for signature in signatures:
    # TODO(Juraj): this behavior should be encapsulated in some utility function
    data_frequency = "T" if "1min" in signature["action_tag"] else "S"
    vendor_name = signature["vendor"].upper()
    mode = "download"
    version = signature["universe"].replace("_", ".")
    exchange_id = signature["exchange_id"]
    universe = imvcounun.get_vendor_universe(vendor_name, mode, version=version)
    universe_list = universe[exchange_id]
    qa_check_list = [
        imvcdqqach.GapsInTimeIntervalBySymbolsCheck(
            config["start_timestamp"], config["end_timestamp"], data_frequency
        ),
        imvcdqqach.NaNChecks(),
        imvcdqqach.OhlcvLogicalValuesCheck(),
        imvcdqqach.FullUniversePresentCheck(universe_list),
        imvcdqqach.DuplicateDifferingOhlcvCheck(),
    ]
    qa_check_lists.append(qa_check_list)

### Cross dataset checks

In [18]:
cross_qa_check_list = [imvcdqqach.OuterCrossOHLCVDataCheck()]

## Initialize QA validators

In [19]:
dataset_validator1 = imvcdqdava.DataFrameDatasetValidator(qa_check_lists[0])
dataset_validator2 = imvcdqdava.DataFrameDatasetValidator(qa_check_lists[1])
cross_dataset_validator = imvcdqdava.DataFrameDatasetValidator(
    cross_qa_check_list
)

## Run QA

In [21]:
status = "SUCCESS"
_LOG.info("First dataset QA:")
error_msgs = dataset_validator1.run_all_checks([data1], abort_on_error=False)
if error_msgs:
    _LOG.info(error_msgs)
    status = "FAILED"
_LOG.info("Second dataset QA:")
error_msgs = dataset_validator1.run_all_checks([data2], abort_on_error=False)
if error_msgs:
    _LOG.info(error_msgs)
    status = "FAILED"
_LOG.info("Cross dataset QA:")
error_msgs = cross_dataset_validator.run_all_checks(datasets, abort_on_error=False)
if error_msgs:
    _LOG.info(error_msgs)
    status = "FAILED"
# If no exception was raised mark the QA as successful.
data_qa_outcome = status

INFO  First dataset QA:
INFO  Running all QA checks:
INFO  	GapsInTimeIntervalBySymbolsCheck: PASSED
INFO  	NaNChecks: PASSED
INFO  	OhlcvLogicalValuesCheck: PASSED
INFO  	FullUniversePresentCheck: PASSED
INFO  	DuplicateDifferingOhlcvCheck: FAILED: Duplicate table contents:
	         timestamp    open    high     low   close  volume currency_pair exchange_id
186  1704314520000  315.72  315.94  315.53  315.71  493.17      BNB_USDT     binance
187  1704314520000  315.72  315.94  315.53  315.75  509.53      BNB_USDT     binance
INFO  Second dataset QA:
INFO  Running all QA checks:
INFO  	GapsInTimeIntervalBySymbolsCheck: PASSED
INFO  	NaNChecks: PASSED
INFO  	OhlcvLogicalValuesCheck: PASSED
INFO  	FullUniversePresentCheck: PASSED
INFO  	DuplicateDifferingOhlcvCheck: PASSED
INFO  Cross dataset QA:
INFO  Running all QA checks:
INFO  	OuterCrossOHLCVDataCheck: FAILED: Different data found:
	         timestamp  open_A  high_A   low_A  close_A  volume_A currency_pair exchange_id_A  open_B  hi

In [22]:
if data_qa_outcome == "FAILED":
    hdbg.dfatal("QA Check unsuccessful for atleast one of the dataset")

AssertionError: 
################################################################################
QA Check unsuccessful for atleast one of the dataset
################################################################################


In [23]:
# This can be read by the invoke task to find out if QA was successful.
hio.to_file("/app/ck_data_reconciliation_outcome.txt", data_qa_outcome)